# **# Pre-processing News Data**

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [ ]:
pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau

- data/news_vnexpress/

    - Kinh tế:
        - bài báo 1.txt
        - bài báo 2.txt
    - Pháp luật
        - bài báo 3.txt
        - bài báo 4.txt

In [ ]:
INPUT ='/content/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [ ]:
import zipfile
# Đường dẫn đến file zip (thay 'your_file.zip' bằng tên file zip bạn đã tải lên)
zip_file = 'news_vnexpress.zip'

# Giải nén file zip
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')

n = 0
for label in os.listdir(INPUT):
    label_path = os.path.join(INPUT, label)

    # Kiểm tra xem đó có phải là thư mục không
    if os.path.isdir(label_path):
        # Đếm số lượng file trong thư mục con
        num_files = len([f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))])
        print(f'{label}: {num_files}')
        n += num_files

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
thoi-su: 59
doi-song: 120
giai-tri: 201
khoa-hoc: 144
kinh-doanh: 262
the-thao: 173
giao-duc: 105
phap-luat: 59
suc-khoe: 162
du-lich: 54
-------------------------
Tổng số văn bản: 1339


In [ ]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['/content/news_vnexpress/khoa-hoc/00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [ ]:
# load dữ liệu các stopwords
stopword = ['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh',
'anh_ấy']

stopwords = [x.strip().replace("_", " ") for x in stopword]
vectorizer_without_stopwords = CountVectorizer()
vectorizer_without_stopwords.fit(data_train.data)
vocab = vectorizer_without_stopwords.vocabulary_
nums_of_stopwords = sum(vocab[word] for word in vocab.keys() if word in stopwords)
print(f"Số lượng từ dừng: {nums_of_stopwords}")
print(stopwords)
# Chuyển hoá dữ liệu text về dạng vector TF
#     - loại bỏ từ dừng
#     - sinh từ điển

module_count_vector = CountVectorizer(stop_words=stopwords)

pipeline = Pipeline([
    ('vectorizer', module_count_vector),
    ('tfidf', TfidfTransformer())
])

# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array
data_preprocessed = pipeline.fit_transform(data_train.data)

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan


print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

Số lượng từ dừng: 2476
['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy']


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'lô', 'nấy', 'đó', 'ấy'] not in stop_words.
  warnings.warn(



Số lượng từ trong từ điển: 13108
Kích thước dữ liệu sau khi xử lý: (1339, 13108)
Kích thước nhãn tương ứng: (1339,)


In [ ]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.11843162 0.        ]]
5


In [ ]:
sum(sum(X[100].toarray() != 0))

428

In [ ]:
print(X[100])

  (0, 13106)	0.11843162414954997
  (0, 13032)	0.04318409010538516
  (0, 13022)	0.0289816992496587
  (0, 13013)	0.02101375062808559
  (0, 13005)	0.033179731882307
  (0, 13000)	0.011705168291761488
  (0, 12999)	0.017508728791792248
  (0, 12983)	0.03807257372545188
  (0, 12982)	0.012779456747888558
  (0, 12978)	0.02675675372615028
  (0, 12977)	0.04000600971255874
  (0, 12971)	0.015015225621772483
  (0, 12950)	0.035987232004406254
  (0, 12946)	0.025453356563118226
  (0, 12932)	0.020377869875602708
  (0, 12929)	0.016259863255706847
  (0, 12926)	0.2797263592768559
  (0, 12919)	0.0674435686507365
  (0, 12915)	0.055671052741889375
  (0, 12909)	0.013443487672683833
  (0, 12890)	0.06338967814433386
  (0, 12883)	0.03268120834775442
  (0, 12877)	0.03937801012229831
  (0, 12864)	0.02802162970968281
  (0, 12856)	0.02702855594070607
  :	:
  (0, 2121)	0.02172886026538689
  (0, 2111)	0.022707659194638512
  (0, 2085)	0.012263714907304914
  (0, 2060)	0.0605832856519242
  (0, 2048)	0.03443464303978417
  (